In [1]:
%cd ..

/Users/heilman/galvanize/recipe-vectorizer


In [2]:
from src.functions import *
from src.modeling_functions import *
import numpy as np
import networkx as nx
import pandas as pd
from collections import Counter
from string import *
from sklearn.metrics.pairwise import cosine_similarity
from sys import argv, exit
import pymongo
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import cm

In [3]:
mc = pymongo.MongoClient()
db = mc['allrecipes']
recipes_coll = db['recipes']

In [6]:
recipes, ratings = find_recipes_matching_search(recipes_coll, 'chili')

In [7]:
len(recipes)

296

In [9]:
units_by_ing = get_common_units_by_ingredient(recipes)

In [17]:
units_by_ing['onion'].most_common(1)[0][0]

'each'

In [ ]:
conversion_dict

In [21]:
def convert_qty_to_common_units(ingredient,
                                norm_qty,
                                units_by_ing,
                                conversion_dict=conversion_dict):
    desired_units = units_by_ing[ingredient].most_common(1)[0][0]
    try:
        ingredient_conversions = conversion_dict[desired_units]
        try:
            return norm_qty / ingredient_conversions[ingredient], desired_units
        except:
            return norm_qty / ingredient_conversions['other'], desired_units
    except:
        return norm_qty, desired_units

In [26]:
convert_qty_to_common_units('paprika', .5, units_by_ing)

(3.0, 'teaspoon')